<a href="https://colab.research.google.com/github/vicheda/aboutMe/blob/main/Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Input your name and netid ((double-click on the cell below and type in)

### [your name, your netid]:

## Q0. Preparations.



### Import/install all needed packages here.
Run the code blocks without change anything!


In [ ]:
import json
import pandas as pd
import scipy
from scipy import stats
!pip install apyori
from apyori import apriori
import matplotlib.pyplot as plt

## Download datasets to the current directory

In [ ]:
#yelp_academic_dataset_user_no_friend.json
!wget https://nu-cs326-msai339.s3.us-east-1.amazonaws.com/yelp_academic_dataset_user_no_friend.json

In [ ]:
#yelp_academic_dataset_business.json
!wget https://nu-cs326-msai339.s3.us-east-1.amazonaws.com/yelp_academic_dataset_business.json

## Load data to pandas DataFrames.

In [ ]:
df_user = pd.read_json("yelp_academic_dataset_user_no_friend.json", lines=True)

In [ ]:
df_bussiness = pd.read_json('yelp_academic_dataset_business.json', lines=True)

In [ ]:
# to browse the dataframe df_user
df_user.head()

In [ ]:
# to browse the dataframe df_bussiness
df_bussiness.head()

## Q1. Correlation coefficient

### Q1a

In [ ]:
def Q1a():
  pass

In [ ]:
Q1a()

### Q1b

#### Analysis.

In [ ]:
def Q1b():
  pass

In [ ]:
Q1b()

#### Conclusions.

In [ ]:
#TODO: modify the conclusions strings
def Q1b_concl():
  conclusions = " put your findings here."
  return conclusions

In [ ]:
Q1b_concl()

### Q1c

#### Analysis.

In [ ]:
def Q1c():
  pass

In [ ]:
Q1c()

#### Plot.

In [ ]:
def Q1c_plot():
  pass

In [ ]:
Q1c_plot()

#### Conclusions.

In [ ]:
#TODO: modify the conclusions strings
def Q1c_concl():
  conclusions = " put your findings here."
  return conclusions

In [ ]:
Q1c_concl()

## Q2. Chi Square Test

#### Analysis.

In [ ]:
def Q2():
  pass

In [ ]:
Q2()

#### Conclusions.

In [ ]:
#TODO: modify the conclusions strings
def Q2_concl():
  pass

In [ ]:
Q2_concl()

## Q3. Association rule

In [ ]:
#helper function
def Q3o():
  pass

### Q3a

In [ ]:
def Q3a():
  pass

In [ ]:
Q3a()

### Q3b

#### Analysis.

In [ ]:
def Q3b():
  pass

In [ ]:
Q3b()

#### Conclusions.

In [ ]:
#TODO: modify the conclusions strings
def Q3b_concl():
  pass

In [ ]:
print(Q3b_concl())

## Q4. ANOVA

### Q4a

In [ ]:
#helper function
def Q4o():
  pass

#### Plot.

In [ ]:
def Q4a():
  pass


In [ ]:
Q4a()

#### Conclusions.

In [ ]:
def Q4a_concl():
  pass


In [ ]:
Q4a_concl()

### Q4b

#### Formulate hypothesis.

In [ ]:
def Q4b_hyp():
  Ho=""
  Ha=""
  return H0,Ha


In [ ]:
Q4b_hyp()

#### Check for ANOVA assumptions.

In [ ]:
def Q4b_var():
  pass


In [ ]:
Q4a_var()

In [ ]:
def Q4b_dist():
  pass


In [ ]:
Q4a_dist()

#### ANOVA.

In [ ]:
def Q4b_anova():
  pass


In [ ]:
Q4a_anova()

#### Conclusions.

In [ ]:
def Q4b_concl():
  pass


In [ ]:
Q4b_concl()

### Q4c

#### Formulate hypothesis.

#### Check for ANOVA assumptions.

#### ANOVA.

#### Conclusions.